In [1]:
%load_ext autoreload
%autoreload now

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-08137aa2-e69b-5e74-8390-7997329b1336"

# Download and convert data

In [3]:
from document_segmentation.pagexml.annotations.generale_missiven import GeneraleMissiven
from document_segmentation.settings import (
    GENERALE_MISSIVEN_DOCUMENT_DIR,
    GENERALE_MISSIVEN_SHEET,
)

N = None

sheet = GeneraleMissiven(GENERALE_MISSIVEN_SHEET)
documents = sheet.download(GENERALE_MISSIVEN_DOCUMENT_DIR, N)

# Load Data

In [4]:
%autoreload now

In [5]:
TRAINING_DATA = 0.8

In [6]:
from document_segmentation.model.dataset import DocumentDataset

dataset = DocumentDataset.from_documents(documents)
dataset.shuffle()

len(dataset)

Skipping row with inventory number 1171 due to reason: 'Niet gedigitaliseerd.'


In [ ]:
dataset._class_counts()

Counter({<Label.IN: 1>: 4823, <Label.BEGIN: 0>: 100, <Label.END: 2>: 98})

In [ ]:
dataset.class_weights()

[0.9900990099009901, 0.020729684908789386, 1.0101010101010102, 100.0]

In [ ]:
training_data, test_data = dataset.split(TRAINING_DATA)

In [ ]:
training_data._class_counts()

Counter({<Label.IN: 1>: 3808, <Label.BEGIN: 0>: 80, <Label.END: 2>: 78})

In [ ]:
test_data._class_counts()

Counter({<Label.IN: 1>: 1015, <Label.BEGIN: 0>: 20, <Label.END: 2>: 20})

# Train Model

In [ ]:
BATCH_SIZE = 128
EPOCHS = 10
WEIGHTS = dataset.class_weights()

In [ ]:
from document_segmentation.model.page_sequence_tagger import PageSequenceTagger

tagger = PageSequenceTagger()

In [ ]:
tagger._device

'mps'

In [ ]:
tagger

PageSequenceTagger(
  (_page_embedding): PageEmbedding(
    (_region_model): RegionEmbeddingSentenceTransformer(
      (_transformer_model): SentenceTransformer(
        (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: RobertaModel 
        (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
      )
      (_region_type): Embedding(9, 16)
      (_linear): Linear(in_features=784, out_features=512, bias=True)
    )
    (_rnn): LSTM(512, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
    (_linear): Linear(in_features=512, out_features=256, bias=True)
  )
  (_rnn): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (_linear): Linear(in_features=512, out_feature

In [ ]:
tagger.train_(
    training_data, test_data, epochs=EPOCHS, batch_size=BATCH_SIZE, weights=WEIGHTS
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: carschno. Use `wandb login --relogin` to force relogin


Evaluating: 100%|██████████| 23/23 [00:44<00:00,  1.94s/batch]
        [1],
        [2]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
Evaluating: 100%|██████████| 23/23 [00:00<00:00, 28.21batch/s]
        [1],
        [2]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
Evaluating: 100%|██████████| 23/23 [00:00<00:00, 28.49batch/s]
        [1],
        [2]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
Evaluating: 100%|██████████| 23/23 [00:00<00:00, 24.04batch/s]
        [1],
        [2]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
Evaluating: 100%|██████████| 23/23 [00:00<00:00, 26.43batch/s]
        [1],
        [2]]) classes have zero instances in both the predictions and the ground truth labels. Preci

MulticlassAccuracy,▁▁▁▁▁▁▁▁▁▁
batch_size,▅█▄▂▁▃▁▂▄▁▂▂▂█▁▂▁▁▁▇█▁▂▁▁▂▄▁▅▁█▃▅▁▁▃▄▃▁▃
loss,▁▁▁▂▅▁▃▂▁▃▃▂▂▁▅▂▃▄▆▁▁▅▂▃█▂▁▄▁█▁▁▁█▃▂▁▁▆▂
MulticlassAccuracy,0.0
batch_size,10
loss,0.37769


# Evaluate Model

In [ ]:
import sys

precision, recall, f1, accuracy = tagger.eval_(test_data, BATCH_SIZE, sys.stdout)

Predicted	Actual	Page ID	Text	Scores


Evaluating:  22%|██▏       | 5/23 [00:00<00:01, 15.44batch/s]

OUT	BEGIN	NL-HaNA_1.04.02_1082_0287.jpg	Het onse brieven vanden 28en december, 1623: p d'e	[1.0550788829277735e-06, 1.775759528754861e-06, 3.6215681120665977e-06, 0.9999935626983643]
OUT	IN	NL-HaNA_1.04.02_1082_0288.jpg	noop der retoucx van; Middelb:; Gouda; Hollandia.;	[1.4826386518507206e-07, 1.6882584930044686e-07, 2.0035014358654735e-07, 0.9999995231628418]
OUT	IN	NL-HaNA_1.04.02_1082_0289.jpg	Eurat e Marcia. handel, hebben wij op 9=en Aug:tso	[1.0185276266838628e-07, 1.0159315877444897e-07, 1.0287053697766169e-07, 0.9999996423721313]
OUT	IN	NL-HaNA_1.04.02_1082_0290.jpg	haer Persien maecken wij reeck. e dat van; suratte	[9.734942807426705e-08, 9.31537158521678e-08, 9.121684740875935e-08, 0.9999996423721313]
OUT	IN	NL-HaNA_1.04.02_1082_0291.jpg	van dien soo sullen wij genootsaect wesen, t' geen	[9.621307128782064e-08, 9.087769115012634e-08, 8.79604655779076e-08, 0.9999996423721313]
OUT	IN	NL-HaNA_1.04.02_1082_0292.jpg	wt Bedalt. te; dop vitsix; consorphe der; speretje	[9.3373948573

Evaluating:  35%|███▍      | 8/23 [00:00<00:01, 14.06batch/s]

OUT	BEGIN	NL-HaNA_1.04.02_1071_0175.jpg	Ed: Erntfeste. Wijse. Voorsinnige, seer discrete; 	[1.8986028749168327e-07, 2.4636574380565435e-07, 3.5447712321001745e-07, 0.9999991655349731]
OUT	IN	NL-HaNA_1.04.02_1071_0176.jpg	tegens Bantam  den; Mataram.; Sampson, Zeewolff & 	[4.20949710644436e-08, 4.51356463315733e-08, 4.747583659536758e-08, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1071_0177.jpg	nilla geweest Ao. 11620.; goee quartiteijt pepar v	[3.4315196018042116e-08, 3.519461344581032e-08, 3.470982434805592e-08, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1071_0178.jpg	jacht Setrate met nagelen; wt Matico ende Anvogno.	[3.226425349112105e-08, 3.3429813584007206e-08, 3.3785827469046126e-08, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1071_0179.jpg	van Nagelen in Prolnco &; Amboyno.; voornemen vand	[3.238546497641437e-08, 3.3573968494238215e-08, 3.3556553091784735e-08, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1071_0180.jpg	1/v depenplip vels plaetsen.; Goun: Spuelt vetsame	[3.2575535158

...

OUT	BEGIN	NL-HaNA_1.04.02_1135_0025.jpg	dele, Erntfeste, Wijse, voorsienige.; ende seer di	[1.9005902629487537e-07, 2.4688080202395213e-07, 3.5749044968724775e-07, 0.9999991655349731]
OUT	IN	NL-HaNA_1.04.02_1135_0026.jpg	de Buijs compt; over Maúritiús.; t'welck den almog	[4.174532364231709e-08, 4.565645639331706e-08, 4.9093998200078204e-08, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1135_0027.jpg	18. - pr. d'Engelsc hoopwel.; is Copie van brieff;	[3.315612318033345e-08, 3.4800375914301185e-08, 3.5401033215975986e-08, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1135_0028.jpg	petten arriveert; mede over dat; Eijlandt.; Berckh	[3.270465143145884e-08, 3.3747056704669376e-08, 3.4256078862426875e-08, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1135_0029.jpg	vander stel ende; ilviús leggen; dsaeckoverhoop,; 	[3.346445254237551e-08, 3.35363985470849e-08, 3.352341693130256e-08, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1135_0030.jpg	becomen; Ebbenhout; ende wat daer; resteert.; de f	[3.414134042

Evaluating:  70%|██████▉   | 16/23 [00:01<00:00, 12.63batch/s]

OUT	BEGIN	NL-HaNA_1.04.02_1104_0009.jpg	Erntteste, wijse, Voorsienige, ende seer discrete 	[1.151914489128103e-06, 1.9648318811960053e-06, 4.095822987437714e-06, 0.999992847442627]
OUT	IN	NL-HaNA_1.04.02_1104_0010.jpg	spoedich suijd te winnen, om te eerder inde passaa	[1.576918293721974e-07, 1.8039052918084053e-07, 2.1820262929850287e-07, 0.9999994039535522]
OUT	IN	NL-HaNA_1.04.02_1104_0011.jpg	arriuent den 4 Sepembr.; 1632 voor batavia,; ordre	[1.0438214559371772e-07, 1.0407202921669523e-07, 1.0624618340671077e-07, 0.9999996423721313]
OUT	IN	NL-HaNA_1.04.02_1104_0012.jpg	voorgenomen reijse; Shertognbosch Leijen Ta.; Phil	[9.78457066480587e-08, 9.36228872205902e-08, 9.1874944985193e-08, 0.9999996423721313]
OUT	IN	NL-HaNA_1.04.02_1104_0013.jpg	dhi gul. brouwer; gaet en 7 sepembr. met; sijn aut	[9.523972721581231e-08, 9.017202273753355e-08, 8.711795373983477e-08, 0.9999996423721313]
OUT	IN	NL-HaNA_1.04.02_1104_0014.jpg	Cabrel bona espiranc; te vruersche.; d Sandt van; 	[9.528460509500292

Evaluating: 100%|██████████| 23/23 [00:01<00:00, 14.42batch/s]

OUT	IN	NL-HaNA_1.04.02_1107_0231.jpg	dat men trachten sab; te accommoderen.; vreemde mi	[3.352961996938575e-08, 3.308101170773625e-08, 3.277721560834834e-08, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1107_0232.jpg	Rijs uijt Middelburch; geloot ende teper in; plaet	[3.494124811709298e-08, 3.500455036942185e-08, 3.397789072323576e-08, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1107_0233.jpg	Resterende; getrocken wisselen; trocken wisselen w	[4.754923921268528e-08, 4.5631381340172084e-08, 4.1061685607246545e-08, 0.9999998807907104]
OUT	IN	NL-HaNA_1.04.02_1107_0234.jpg	vorder Exploicten; en tochten op Ban„; tams Onderd	[3.4047755548272107e-07, 2.4276741328321805e-07, 1.4880181709031604e-07, 0.9999992847442627]
OUT	IN	NL-HaNA_1.04.02_1107_0235.jpg	geruchten tot; Bantam.; met een dolpsijn in; aerij	[2.627134847443813e-07, 3.311849923193222e-07, 4.4457905801209563e-07, 0.999998927116394]
OUT	END	NL-HaNA_1.04.02_1107_0236.jpg	Eijlanden met schoon ende bequaem weder, dat naer 	[3.9800463014216

In [ ]:
import csv

from document_segmentation.pagexml.datamodel.label import Label

writer = csv.DictWriter(
    sys.stdout, fieldnames=["Metric"] + [label.name for label in Label], delimiter="\t"
)

writer.writeheader()

for metric in precision, recall, f1:
    assert metric.average is None

    scores: list[float] = metric.compute().tolist()
    writer.writerow(
        {"Metric": metric.__class__.__name__}
        | {label.name: f"{score:.4f}" for label, score in zip(Label, scores)}
    )

assert accuracy.average is not None

print(
    f"{accuracy.__class__.__name__} ({accuracy.average} average):\t{accuracy.compute().item():.4f}",
    file=sys.stdout,
)

        [1],
        [2]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.


Metric	BEGIN	IN	END	OUT
MulticlassPrecision	0.0000	0.0000	0.0000	0.0000
MulticlassRecall	0.0000	0.0000	0.0000	0.0000
MulticlassF1Score	0.0000	0.0000	0.0000	0.0000
MulticlassAccuracy (micro average):	0.0000
